# BayesLOSVD and DYNAMITE

This notebook will show how to run DYNAMITE orbit-based models using LOSVDs extracted using [BayesLOSVD](https://github.com/jfalconbarroso/BAYES-LOSVD). This is a Python library for the non-parametric extraction of the Line-Of-Sight Velocity Distributions in galaxies. Rather than describe the LOSVD using a Gauss-Hermite expansion, BayesLOSVD directly constrains the LOSVD in histogrammed velocity bins.

## The galaxy

We will model the galaxy NGC4550. The following is taken from Figure 7 of Falcón-Barroso & Martig 2020, showing LOSVDs extracted using BayesLOSVD at three regions,

<img src="NGC4550_input/bayeslosvd_extractions.png" width="400">

The galaxy has clearly bimodal LOSVD, indicating a strong counter-rotating stellar component.

## Data preparation

We require the ``*_results.hdf5`` output file from BayesLOSVD. Two such files for NGC4550 have been kindly provided by Jesus Falcón-Barroso, and can be found in the directory `NGC4550_input`. They differ in the form of LOSVD regularisation used,
- `NGC4550_SAURON-SP_results.hdf5`: using simplex regularisation
- `NGC4550_SAURON-RW_results.hdf5`: using random walk regularisation

For details, and instructions on creating your own ``*_results.hdf5`` files, see the [BayesLOSVD](https://github.com/jfalconbarroso/BAYES-LOSVD) documentation.

For use in DYNAMITE, we must convert this file to the Astropy ECSV format 

In [1]:
import dynamite as dyn
print('Using DYNAMITE version:', dyn.__version__)
print('Located at:', dyn.__path__)
    
BayesLOSVD = dyn.kinematics.BayesLOSVD(weight=1.,
                                       hist_width=1,
                                       hist_center=0,
                                       hist_bins=0,
                                       type='BayesLOSVD')
infile = 'NGC4550_input/NGC4550_SAURON-SP_results.hdf5'
outfile = 'NGC4550_input/dynamite_input/bayes_losvd_kins.ecsv'
BayesLOSVD.write_losvds_to_ecsv_format(infile, outfile=outfile)

We next add the PSF to the kinematics. The seeing - from Table 3 of [Emsellem et al 2004](https://academic.oup.com/mnras/article/352/3/721/1210712) - has FWHM of 2.1 arcsec. We convert this to a Gaussian sigma, and add to the header of the kinematics file,

In [2]:
seeing_fwhm = 2.1
seeing_gauss_sigma = seeing_fwhm/2.35
# add the psf to file header
BayesLOSVD.add_psf_to_datafile(sigma=[seeing_gauss_sigma],
                               weight=[1.],
                               datafile=outfile)

# re-create the BayesLOSVD object reading in the complete kinematics file
BayesLOSVD = dyn.kinematics.BayesLOSVD(datafile=outfile,
                                       weight=1.,
                                       type='BayesLOSVD')

Next create the auxillary `aperture.dat` and `bins.dat` files. This will require the galaxy's position angle. Table 3 of [Emsellem et al 2004](https://academic.oup.com/mnras/article/352/3/721/1210712), gives PA = 0.

In [3]:
position_angle = 0.
angle_deg = 90. - position_angle
dyn_input_direc = 'NGC4550_input/dynamite_input/'
BayesLOSVD.write_aperture_and_bin_files(filename=infile,
                                        angle_deg=angle_deg,
                                        center='max_flux',
                                        aperture_filename=dyn_input_direc+'aperture.dat',
                                        bin_filename=dyn_input_direc+'bins.dat')

Finally we need the Multi Gaussian Expansion. MGEs for Atlas3D galaxies (including NGC4550) can be found in the `MGE parameters for the deconvolved r-band surface brightness` link of the [Atlas3D page](http://www-astro.physics.ox.ac.uk/atlas3d/). The MGE data for NGC4550 is provided here in the required Astropy ECSV format, ``NGC4550_input/dynamite_input/mge.ecsv``

The input directory `NGC4550_input` now contains all 4 required files:
- ``mge_NGC4550.ecsv``
- ``NGC4550_bayeslovd_kins.ecsv``
- ``aperture.dat``
- ``bins.dat``

## Preparing the configuration file

Next prepare the DYNAMITE congfiguration file. We've included the file `NGC4550_config.yaml`,

In [6]:
c = dyn.config_reader.Configuration('NGC4550_config.yaml', reset_logging=True)

[INFO] 14:09:59 - dynamite.config_reader.Configuration - Config file NGC4550_config.yaml read.
[INFO] 14:09:59 - dynamite.config_reader.Configuration - io_settings...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - system_attributes...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - model_components...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - system_parameters...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - orblib_settings...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - weight_solver_settings...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - parameter_space_settings...
[INFO] 14:09:59 - dynamite.config_reader.Configuration - legacy_settings...
[ERROR] 14:09:59 - dynamite.physical_system.System - System needs distMPc, name, and position_angle attributes


ValueError: System needs distMPc, name, and position_angle attributes

In [ ]:
%debug

> /Users/pjethwa/miniconda3/envs/dyn_env_py37/lib/python3.7/site-packages/dynamite-0.0.0-py3.7.egg/dynamite/physical_system.py(61)validate()
     59             text = 'System needs distMPc, name, and position_angle attributes'
     60             self.logger.error(text)
---> 61             raise ValueError(text)
     62         if not self.cmp_list:
     63             text = 'System has no components'

ipdb> self.distMPc
15.5
ipdb> self.name
'NGC4550'
ipdb> self.position_angle
0.0
